In [1]:
from preamble import *
from scipy.stats import pearsonr

In [2]:
pdo = pd.read_csv('Data/PDO/data_PDO.csv')

In [3]:
duration = pd.read_csv('Data/computed-data/computed_annual-intensification-duration_JTWC.csv')

In [4]:
pdo.head()

,month,YEAR,PDO,TIME
0,1,1951,-1.54,1951-01-01
1,1,1952,-2.01,1952-01-01
2,1,1953,-0.57,1953-01-01
3,1,1954,-1.32,1954-01-01
4,1,1955,0.20,1955-01-01


### Monthly Correlation

In [5]:
pdo_correlation = []

for i in (np.arange(12)+1):
    # Extract pdo temperature for exactly one month each year
    temp = pdo[pdo['month']==i]['PDO'].values
    # Calculate correlation with duration
    pdo_correlation.append(pearsonr(duration['Intensification Duration'],temp)[0])

In [6]:
corr_table = pd.DataFrame({'month':(np.arange(12)+1),'corr_pdo':pdo_correlation})

In [7]:
monthly_corr = corr_table.set_index('month')
monthly_corr.round(2).transpose()

month,1,2,3,4,5,6,7,8,9,10,11,12
corr_pdo,0.25,0.15,0.2,0.23,0.18,0.18,0.19,0.29,0.28,0.26,0.24,0.3


### Quarterly Correlation

In [8]:
q_pdo_correlation = []

for i in (np.arange(4)):
    # Extract pdo temperature for each quarter each year
    temp = pdo[pdo['month'].between(3*i+1,3*(i+1))].groupby('YEAR').mean()['PDO'].values
    # Calculate correlation with duration
    q_pdo_correlation.append(pearsonr(duration['Intensification Duration'],temp)[0])

In [9]:
q_corr_table = pd.DataFrame({'quarter':['Jan-Mar','Apr-Jun','Jul-Sep','Oct-Dec'],'corr_pdo':q_pdo_correlation})

In [10]:
q_corr = q_corr_table.set_index('quarter')
q_corr.round(2).transpose()

quarter,Jan-Mar,Apr-Jun,Jul-Sep,Oct-Dec
corr_pdo,0.21,0.21,0.28,0.29


### Annual Correlation

In [11]:
y_pdo_correlation = []

# Calculate correlation of annual pdo temperature with duration
print('PDO correlation coefficient: {:.3f}'.format(pearsonr(duration['Intensification Duration'],
                                                pdo.groupby('YEAR').mean()['PDO'].values)[0]))

PDO correlation coefficient: 0.303


In [12]:
pearsonr(duration['Intensification Duration'],pdo[pdo['month'].between(7,10)].groupby('YEAR').mean()['PDO'].values)[0]

0.28458909209749816

In [ ]:
from sklearn.preprocessing import normalize
pdo_mean = pdo_test['PDO'].mean()
pdo_std = pdo_test['PDO'].std()
pdo_test['norm_pdo'] = (pdo_test['PDO']-pdo_mean)/pdo_std
pdo_test.plot('YEAR','norm_pdo')
plt.show()
pdo_test['norm_pdo'] = normalize(pdo_test['PDO'][np.newaxis,:])[0]